In [2]:
# ===========================================================
# End-to-end: Predict -> Export -> TrackEval -> Official metrics
# ===========================================================

# ---- numpy compatibility shim (put at top, BEFORE importing trackeval) ----
import numpy as np

# Add removed numpy aliases back (safe: maps to built-in types)
# Only add the ones you need to avoid masking real problems
if not hasattr(np, "float"):
    np.float = float
if not hasattr(np, "int"):
    np.int = int
if not hasattr(np, "bool"):
    np.bool = bool
# ---------------------------------------------------------------------------

import os
from pathlib import Path
import json
# import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from collections import defaultdict
from textwrap import dedent # ADDED for minimal yaml generation

# --- CONFIG (edit paths as needed) ---
IMAGE_DIR   = Path(r"C:\Users\Manish\Documents\SpikeYOLOPaper\KITTITRACK\data_tracking_image_2\training\image_02")
GT_ROOT     = Path(r"C:\Users\Manish\Documents\SpikeYOLOPaper\KITTITRACK\data_tracking_label_2\training")  # contains label_02/
LABEL_DIR   = GT_ROOT / "label_02"
OUTPUT_DIR  = Path(r"C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking") # CHANGED OUTPUT_DIR for cleaner separation

MODEL_PATH  = Path("KITTI.pt")
TRACKER_YAML= "botsort.yaml"  # CHANGED to botsort.yaml
CONF_THRESH = 0.1
SAVE_VIDEOS = True
TRACKER_NAME= "SpikeYOLO"

# Optional: limit sequences for a quick pass (None = all)
MAX_SEQUENCES = None    # e.g., 3

# --- Create folders ---
(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "videos").mkdir(exist_ok=True)
(OUTPUT_DIR / "predictions").mkdir(exist_ok=True)
(OUTPUT_DIR / "metrics").mkdir(exist_ok=True)

# TrackEval IO layout
TE_ROOT           = OUTPUT_DIR / "trackeval_io"
TE_TRACKERS_ROOT = TE_ROOT / "trackers" / "KITTI_2D_BOX"
TE_TRKDATA       = TE_TRACKERS_ROOT / TRACKER_NAME / "data"
TE_TRKDATA.mkdir(parents=True, exist_ok=True)

RAW_OUT = OUTPUT_DIR / "metrics" / "trackeval_raw"
RAW_OUT.mkdir(parents=True, exist_ok=True)

SEQMAP_FILE = GT_ROOT / "evaluate_tracking.seqmap.training"

# --- Imports that can be heavy ---
from ultralytics import YOLO
import trackeval
from scipy.optimize import linear_sum_assignment  # for any fallback matching (not strictly needed now)

# ========== 1) Utilities ==========

CLASS_MAPPING = {
    'Car': 'Car', 'Van': 'Car', 'Truck': 'Car', 'Tram': 'Car',
    'Pedestrian': 'Pedestrian', 'Person_sitting': 'Pedestrian',
    'Cyclist': 'IGNORE', 'Misc': 'IGNORE', 'DontCare': 'IGNORE'
}
VALID_CLASSES = ['Car', 'Pedestrian']

def get_sequences(image_dir: Path):
    seqs = sorted([d.name for d in image_dir.iterdir() if d.is_dir()])
    if MAX_SEQUENCES is not None:
        seqs = seqs[:MAX_SEQUENCES]
    print(f"Found {len(seqs)} sequences: {seqs[:5]}{' ...' if len(seqs)>5 else ''}")
    return seqs

def setup_model_mapping(model):
    """Map model classes to KITTI classes."""
    model_to_kitti = {}
    target_ids       = []
    print("Detected model classes:", model.names)
    for cls_id, name in model.names.items():
        n = name.lower().strip()
        if n in ['car','van','truck','tram','vehicle']:
            model_to_kitti[cls_id] = 'Car'
            target_ids.append(cls_id)
        elif n in ['pedestrian','person','person_sitting']:
            model_to_kitti[cls_id] = 'Pedestrian'
            target_ids.append(cls_id)
    print("Class mapping (model -> KITTI):", {model.names[i]:model_to_kitti[i] for i in model_to_kitti})
    return model_to_kitti, target_ids

def load_kitti_labels(sequence_id: str):
    file = LABEL_DIR / f"{sequence_id}.txt"
    if not file.exists():  
        return None
    rows = []
    with open(file, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 10:  
                continue
            frame = int(p[0]); tid = int(p[1]); obj = p[2]
            if obj not in CLASS_MAPPING:  
                continue
            mapped = CLASS_MAPPING[obj]
            if mapped == "IGNORE":  
                continue
            x1,y1,x2,y2 = map(float, p[6:10])
            rows.append({
                "frame": frame, "id": tid, "type": mapped,
                "x": x1, "y": y1, "w": x2-x1, "h": y2-y1
            })
    df = pd.DataFrame(rows)
    return df

def export_seq_to_kitti_result(pred_df: pd.DataFrame, sequence_id: str):
    """
    KITTI 2D tracking results format (per-seq file):
    frame id type truncated occluded alpha x1 y1 x2 y2 h w l X Y Z ry score
    Unknown 3D fields filled with sentinels.
    """
    out = TE_TRKDATA / f"{sequence_id}.txt"
    rows = []
    for _, r in pred_df.iterrows():
        x1 = float(r['x']); y1 = float(r['y'])
        x2 = float(r['x'] + r['w']); y2 = float(r['y'] + r['h'])
        rows.append([
            int(r['frame']), int(r['id']), str(r['type']),
            -1, -1, -10, x1, y1, x2, y2,
            -1, -1, -1, -1000, -1000, -1000, -10,
            float(r.get('conf', 1.0))
        ])
    rows.sort(key=lambda R: (R[0], R[1]))
    with out.open("w") as f:
        for R in rows:
            f.write(
                f"{R[0]} {R[1]} {R[2]} {R[3]} {R[4]} {R[5]} "
                f"{R[6]:.2f} {R[7]:.2f} {R[8]:.2f} {R[9]:.2f} "
                f"{R[10]} {R[11]} {R[12]} {R[13]} {R[14]} {R[15]} {R[16]} {R[17]:.6f}\n"
            )
    return out

def build_seqmap_for_present_sequences():
    """
    Build seqmap only for sequences that have BOTH GT and tracker files.
    """
    trk_seqs = sorted(p.stem for p in TE_TRKDATA.glob("*.txt"))
    gt_seqs  = sorted(p.stem for p in LABEL_DIR.glob("*.txt"))
    present  = sorted(set(trk_seqs).intersection(gt_seqs))
    if not present:
        raise RuntimeError("No overlapping sequences between tracker results and GT.")
    rows = []
    for seq in present:
        gt_file = LABEL_DIR / f"{seq}.txt"
        df = pd.read_csv(gt_file, sep=r"\s+", header=None, usecols=[0], engine="python")
        if df.empty:  
            continue
        max_f = int(df.iloc[:,0].max())
        rows.append((seq, 0, max_f+1))
    if not rows:
        raise RuntimeError("No non-empty GT sequences to evaluate.")
    with SEQMAP_FILE.open("w") as f:
        for seq, s, e in rows:
            f.write(f"{seq} empty {s:06d} {e:06d}\n")
    print(f"✓ Wrote seqmap with {len(rows)} sequences at: {SEQMAP_FILE}")

def _color_for_class(class_name: str, track_id: int):
    np.random.seed(track_id)
    base = np.array([0, 255, 0]) if class_name == 'Car' else np.array([255, 100, 0])
    return tuple(map(int, np.clip(base + np.random.randint(-30, 30, 3), 0, 255)))

# ADDED utility to ensure correct video format
def _ensure_bgr_uint8(img):
    if img is None:
        return None
    if img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)
    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    elif img.ndim == 3 and img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img

# ========== 2) Predict all sequences (YOLO+BOTSort; stream + GMC disabled; skip processed) ==========
print("\n=== Running predictions (YOLO + BOTSort) ===")
sequences = get_sequences(IMAGE_DIR)
model = YOLO(str(MODEL_PATH))
model_to_kitti, target_ids = setup_model_mapping(model)

# --- Prepare a temporary tracker YAML with GMC turned off ---
tmp_tracker_yaml = OUTPUT_DIR / "botsort_no_gmc.yaml"
trk_path = Path(TRACKER_YAML)

# These parameters are required by the BoT-SORT class constructor, even if not used (due to `with_reid: false`)
BOT_SORT_DEFAULTS = """
proximity_thresh: 0.5
appearance_thresh: 0.25
"""

# The parameter the tracker class looks for is the flat key 'gmc_method'
GMC_OVERRIDE = "gmc_method: none"


if trk_path.exists():
    # Read original and modify it
    with open(trk_path, "r") as f:
        orig_yaml = f.read()

    # 1. Ensure required ReID defaults are present (fixes prior error)
    new_yaml = orig_yaml
    for line in BOT_SORT_DEFAULTS.strip().split('\n'):
        key = line.split(':')[0].strip()
        if f"{key}:" not in new_yaml:
            new_yaml += f"\n{line}"

    # 2. Add or overwrite the flat 'gmc_method' parameter for KITTI compatibility
    lines = new_yaml.splitlines()
    found_gmc_method = False
    final_lines = []
    
    for ln in lines:
        if ln.strip().startswith("gmc_method:"):
            # Overwrite existing flat gmc_method
            final_lines.append(GMC_OVERRIDE)
            found_gmc_method = True
        elif ln.strip().startswith("gmc:"):
            # Skip the nested 'gmc:' structure if it exists, as we use the flat key
            continue
        else:
            final_lines.append(ln)

    new_yaml = "\n".join(final_lines)

    # If gmc_method was never found, append it
    if not found_gmc_method:
        new_yaml += f"\n{GMC_OVERRIDE}\n"

else:
    # No tracker YAML provided -> generate a minimal, sane BoT-SORT config with GMC off
    new_yaml = dedent(f"""
        tracker_type: botsort
        track_high_thresh: 0.1
        track_low_thresh: 0.05
        new_track_thresh: 0.2
        match_thresh: 0.8
        track_buffer: 30
        with_reid: false
        # REQUIRED to fix the previous AttributeError
        proximity_thresh: 0.5
        appearance_thresh: 0.25
        # REQUIRED to fix the current AttributeError and disable GMC for KITTI
        {GMC_OVERRIDE}
    """).strip()

with open(tmp_tracker_yaml, "w") as f:
    f.write(new_yaml)

print(f"Using tracker config: {tmp_tracker_yaml}")

all_pred_files = []
for seq_id in sequences:
    print(f"\n--- Sequence {seq_id} ---")
    seq_dir = IMAGE_DIR / seq_id
    imgs = sorted(seq_dir.glob("*.png"))
    if not imgs:
        print("  (no images) -> skip")
        continue

    # --- Skip if already processed ---
    pred_csv_path = OUTPUT_DIR / "predictions" / f"{seq_id}_predictions.csv"
    te_file_path  = TE_TRKDATA / f"{seq_id}.txt"
    video_path    = OUTPUT_DIR / "videos" / f"{seq_id}_tracked.mp4"
    already_done  = pred_csv_path.exists() and te_file_path.exists() and (not SAVE_VIDEOS or video_path.exists())
    if already_done:
        print("  ✓ Already processed (found predictions + TrackEval file"
              f"{' + video' if SAVE_VIDEOS else ''}). Skipping.")
        # keep list for downstream checks (so Step 3/4 can still run)
        all_pred_files.append(pred_csv_path)
        continue

    # Reset any lingering tracker state (IMPORTANT for sequence-by-sequence tracking)
    try:
        if hasattr(model, "predictor") and hasattr(model.predictor, "reset_tracking"):
            model.predictor.reset_tracking()
    except Exception:
        pass

    # Probe first frame to set video size
    probe = _ensure_bgr_uint8(cv2.imread(str(imgs[0]), cv2.IMREAD_COLOR))
    if probe is None:
        print(f"  (cannot read first frame: {imgs[0]}) -> skip")
        continue
    ref_h, ref_w = probe.shape[:2]

    # Video writer
    vw = None
    if SAVE_VIDEOS:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        vw = cv2.VideoWriter(str(video_path), fourcc, 10, (ref_w, ref_h))

    preds = []
    frame_idx = -1

    # Single efficient streaming call over the folder (REPLACED per-frame loop)
    for res in tqdm(
        model.track(
            source=str(seq_dir),
            conf=CONF_THRESH,
            iou=0.5,
            tracker=str(tmp_tracker_yaml),          # GMC disabled
            classes=target_ids if target_ids else None,
            persist=False,
            verbose=False,
            stream=True
        ),
        total=len(imgs),
        desc=f"Tracking {seq_id}"
    ):
        frame_idx += 1
        frame = _ensure_bgr_uint8(res.orig_img)
        if frame.shape[:2] != (ref_h, ref_w):
            frame = cv2.resize(frame, (ref_w, ref_h), interpolation=cv2.INTER_LINEAR)

        boxes = res.boxes
        if boxes is not None and boxes.id is not None:
            xyxy = boxes.xyxy.cpu().numpy()
            tids = boxes.id.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            clss = boxes.cls.cpu().numpy()
            for box, tid, conf, cls_idx in zip(xyxy, tids, confs, clss):
                x1, y1, x2, y2 = box
                k = int(cls_idx)
                if k not in model_to_kitti:
                    continue
                kitti_cls = model_to_kitti[k]
                preds.append({
                    "frame": frame_idx,
                    "id": int(tid),
                    "type": kitti_cls,
                    "x": float(x1),
                    "y": float(y1),
                    "w": float(x2 - x1),
                    "h": float(y2 - y1),
                    "conf": float(conf)
                })
                if SAVE_VIDEOS:
                    color = _color_for_class(kitti_cls, int(tid))
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                    label = f"{kitti_cls} ID:{int(tid)} {conf:.2f}"
                    cv2.putText(frame, label, (int(x1), int(y1) - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        if SAVE_VIDEOS and vw is not None:
            vw.write(frame)

    if SAVE_VIDEOS and vw is not None:
        vw.release()
        print(f"  ✓ Video saved: {video_path}")

    pred_df = pd.DataFrame(preds)
    if pred_df.empty:
        print("  (no predictions) -> skip export")
        continue

    # Save predictions CSV
    pred_df.to_csv(pred_csv_path, index=False)
    all_pred_files.append(pred_csv_path)
    print(f"  ✓ Predictions CSV: {pred_csv_path} ({len(pred_df)} dets)")

    # Export to TrackEval KITTI result file
    te_file = export_seq_to_kitti_result(pred_df, seq_id)
    print(f"  ✓ TrackEval file: {te_file}")

if not all_pred_files:
    raise RuntimeError("No predictions produced; check model path / tracker / confidence threshold.")


# ========== 3) Build seqmap (reconcile GT & tracker files) ==========
build_seqmap_for_present_sequences()

# ========== 4) Run TrackEval (official) ==========
print("\n=== Running TrackEval (official KITTI_2D_BOX) ===")
dataset_config = {
    'GT_FOLDER'         : str(GT_ROOT),
    'TRACKERS_FOLDER' : str(TE_TRACKERS_ROOT),  # includes 'KITTI_2D_BOX'
    'OUTPUT_FOLDER'   : str(RAW_OUT),
    'TRACKERS_TO_EVAL': [TRACKER_NAME],
    'BENCHMARK'       : 'KITTI_2D_BOX',
    'SEQMAP_FILE'     : str(SEQMAP_FILE),
    'DO_PREPROC'      : False,
    'CLASSES_TO_EVAL' : ['Car', 'Pedestrian'],
}
eval_config = {
    'USE_PARALLEL': False,
    'NUM_PARALLEL_CORES': 1,
    'PRINT_RESULTS': True,
    'PRINT_ONLY_COMBINED': False,
    'TIME_PROGRESS': False,
    'DISPLAY_LESS_PROGRESS': True,
}
metrics_list = [trackeval.metrics.HOTA(), trackeval.metrics.CLEAR(), trackeval.metrics.Identity()]
evaluator    = trackeval.Evaluator(eval_config)
dataset_list = [trackeval.datasets.Kitti2DBox(dataset_config)]
output_res, _ = evaluator.evaluate(dataset_list, metrics_list)

# ========== 5) Parse nested results and compute official AUCs ==========
# Your TrackEval build nests metrics by class:
# output_res['Kitti2DBox']['SpikeYOLO'][seq]['car' or 'pedestrian'] has:
#    'HOTA': {'HOTA': np.ndarray, 'DetA': np.ndarray, 'AssA': np.ndarray, 'LocA': np.ndarray, ...}
#    'CLEAR': {'MOTA': float, 'MOTP': float, ...}
#    'Identity': {'IDF1': float, 'IDP': float, 'IDR': float}
#
# Official "AUC" for HOTA/DetA/AssA is the MEAN of those arrays across thresholds.

def arr_mean(x):
    try:
        x = np.asarray(x)
        if x.ndim == 0:
            return float(x)
        return float(np.nanmean(x))
    except Exception:
        return None

CLASS_NAME_MAP = {"car": "Car", "pedestrian": "Pedestrian"}

rows = []
for dataset_name, trackers in output_res.items():
    for tracker_name, seqs in trackers.items():
        for seq_name, metrics in seqs.items():
            for cls_key, cls_block in metrics.items():
                if not isinstance(cls_block, dict):
                    continue
                cls_norm = str(cls_key).lower()
                if cls_norm not in CLASS_NAME_MAP:
                    continue
                pretty_cls = CLASS_NAME_MAP[cls_norm]

                # HOTA group
                HOTA_grp = cls_block.get("HOTA", {})
                HOTA_auc = arr_mean(HOTA_grp.get("HOTA"))
                DetA_auc = arr_mean(HOTA_grp.get("DetA"))
                AssA_auc = arr_mean(HOTA_grp.get("AssA"))
                LocA_auc = arr_mean(HOTA_grp.get("LocA")) if "LocA" in HOTA_grp else None

                # CLEAR group
                CLEAR = cls_block.get("CLEAR", {})
                MOTA  = CLEAR.get("MOTA", None)
                MOTP  = CLEAR.get("MOTP", None)
                MOTA  = float(MOTA) if MOTA is not None else None
                MOTP  = float(MOTP) if MOTP is not None else None

                # Identity group
                ID = cls_block.get("Identity", {})
                IDF1 = float(ID.get("IDF1")) if "IDF1" in ID else None
                IDP  = float(ID.get("IDP"))  if "IDP"  in ID else None
                IDR  = float(ID.get("IDR"))  if "IDR"  in ID else None

                rows.append({
                    "Sequence": seq_name,
                    "Class": pretty_cls,
                    "HOTA": HOTA_auc,
                    "DetA": DetA_auc,
                    "AssA": AssA_auc,
                    "LocA": LocA_auc,
                    "MOTA": MOTA,
                    "MOTP": MOTP,
                    "IDF1": IDF1,
                    "IDP": IDP,
                    "IDR": IDR
                })

metrics_df = pd.DataFrame(rows).sort_values(["Sequence", "Class"])
if metrics_df.empty:
    raise RuntimeError("Parsed metrics are empty. The expected nested keys changed; print output_res to inspect.")

# Save per-seq, per-class metrics
per_seq_csv = OUTPUT_DIR / "metrics" / "trackeval_per_sequence_per_class.csv"
metrics_df.to_csv(per_seq_csv, index=False)
print(f"\n✓ Saved per-seq, per-class metrics: {per_seq_csv}")

# ========== 6) Print per-sequence with BOTH classes side-by-side ==========
def print_seq(seq):
    d = metrics_df[metrics_df["Sequence"]==seq]
    if d.empty:  
        return
    cols = ["Class", "HOTA","DetA","AssA","MOTA","MOTP","IDF1"]
    present = ["Class"] + [c for c in cols if c in d.columns and c != "Class"]
    tab = d[present].set_index("Class")
    # Show decimals (0-1 range) like TrackEval default
    print(f"\nSequence {seq} — OFFICIAL metrics (AUC where applicable)")
    display(tab)

print("\n=== Per-sequence OFFICIAL metrics ===")
for seq in sorted(metrics_df["Sequence"].unique()):
    print_seq(seq)

# ========== 7) Class-wise averages (mean ± std across sequences) ==========
agg = (metrics_df
       .groupby("Class")
       .agg({c:["mean","std"] for c in ["HOTA","DetA","AssA","MOTA","MOTP","IDF1","IDP","IDR"] if c in metrics_df.columns})
       .sort_index())
agg.columns = [f"{m}_{stat}" for m,stat in agg.columns]
class_avg_csv = OUTPUT_DIR / "metrics" / "trackeval_class_averages.csv"
agg.to_csv(class_avg_csv)
print("\n=== Class-wise averages (OFFICIAL) ===")
display(agg)
print(f"✓ Saved class averages: {class_avg_csv}")

# Also save compact JSON
class_avg_json = OUTPUT_DIR / "metrics" / "trackeval_class_averages.json"
class_json = {}
for cls, row in agg.iterrows():
    stats = {}
    for m in ["HOTA","DetA","AssA","MOTA","MOTP","IDF1","IDP","IDR"]:
        mean_col = f"{m}_mean"; std_col = f"{m}_std"
        if mean_col in agg.columns:
            stats[m] = {"mean": float(row[mean_col]), "std": float(row.get(std_col, np.nan))}
    class_json[cls] = stats
with open(class_avg_json, "w") as f:
    json.dump(class_json, f, indent=2)
print(f"✓ Saved class averages JSON: {class_avg_json}")

# ========== 8) Dataset-wide combined (macro) ==========
# Average across all (seq, class) rows
combined = metrics_df.drop(columns=["Sequence","Class"]).mean(numeric_only=True)
print("\n=== Combined (macro average over sequences & classes) ===")
print(combined.to_string(float_format=lambda x: f"{x:0.4f}"))

combined_csv = OUTPUT_DIR / "metrics" / "trackeval_combined_macro.csv"
combined.to_csv(combined_csv)
print(f"✓ Saved combined macro averages: {combined_csv}")


=== Running predictions (YOLO + BOTSort) ===
Found 21 sequences: ['0000', '0001', '0002', '0003', '0004'] ...
Detected model classes: {0: 'Car', 1: 'Pedestrian'}
Class mapping (model -> KITTI): {'Car': 'Car', 'Pedestrian': 'Pedestrian'}
Using tracker config: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\botsort_no_gmc.yaml

--- Sequence 0000 ---


Tracking 0000:   0%|                                                                           | 0/154 [00:00<?, ?it/s]C:\Users\Manish\miniconda3\Lib\site-packages\torch\functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4319.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Tracking 0000: 100%|█████████████████████████████████████████████████████████████████| 154/154 [00:19<00:00,  7.83it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\videos\0000_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\predictions\0000_predictions.csv (659 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\0000.txt

--- Sequence 0001 ---


Tracking 0001: 100%|█████████████████████████████████████████████████████████████████| 447/447 [00:52<00:00,  8.49it/s]


  ✓ Video saved: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\videos\0001_tracked.mp4
  ✓ Predictions CSV: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\predictions\0001_predictions.csv (2926 dets)
  ✓ TrackEval file: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\trackeval_io\trackers\KITTI_2D_BOX\SpikeYOLO\data\0001.txt

--- Sequence 0002 ---


...

,HOTA,DetA,AssA,MOTA,MOTP,IDF1
Class,,,,,,
Car,0.793186,0.757654,0.833487,0.832946,0.891068,0.898976
Pedestrian,0.561714,0.598148,0.528528,0.718066,0.816688,0.715262



=== Class-wise averages (OFFICIAL) ===


,HOTA_mean,HOTA_std,DetA_mean,DetA_std,AssA_mean,AssA_std,MOTA_mean,MOTA_std,MOTP_mean,MOTP_std,IDF1_mean,IDF1_std,IDP_mean,IDP_std,IDR_mean,IDR_std
Class,,,,,,,,,,,,,,,,
Car,0.752150,0.178640,0.719239,0.179321,0.790681,0.187463,0.781101,0.216341,0.846969,0.190198,0.857640,0.200171,0.848544,0.209971,0.874238,0.204753
Pedestrian,0.311157,0.260336,0.314747,0.276425,0.325406,0.257247,0.257406,0.694314,0.536141,0.377001,0.424833,0.354572,0.540869,0.428834,0.369773,0.312746


✓ Saved class averages: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\metrics\trackeval_class_averages.csv
✓ Saved class averages JSON: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\metrics\trackeval_class_averages.json

=== Combined (macro average over sequences & classes) ===
HOTA   0.5317
DetA   0.5170
AssA   0.5580
LocA   0.8894
MOTA   0.5193
MOTP   0.6916
IDF1   0.6412
IDP    0.6947
IDR    0.6220
✓ Saved combined macro averages: C:\Users\Manish\Documents\SpikeYOLOPaper\kitti_tracking\metrics\trackeval_combined_macro.csv


<Figure size 640x480 with 0 Axes>

In [7]:
"""
BDD100K MOT Evaluation with Enhanced Hungarian Matching + Detection Metrics
Date: 2025-11-04 13:14:36 UTC
User: manishkolachalamgmail

Evaluate fine-tuned model with:
- Enhanced Hungarian matching (IoU + motion + appearance)
- Class-wise metrics in real-time
- Soft thresholds for better matching
- Temporal smoothing
- TP, FP, FN per video and per class
"""

import os
import cv2
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

from ultralytics import YOLO
from tqdm import tqdm

try:
    import motmetrics as mm
    from scipy.optimize import linear_sum_assignment
    HAS_MOTMETRICS = True
except ImportError:
    HAS_MOTMETRICS = False
    print("[ERROR] motmetrics not installed. Install with: pip install motmetrics scipy")
    exit(1)

CLASS_NAMES = ["car", "pedestrian"]

# Prediction colors (bright)
PRED_COLORS = {
    0: (0, 255, 0),      # Car predictions - Bright Green
    1: (255, 0, 255),    # Pedestrian predictions - Magenta
}

# Ground truth colors (different from predictions)
GT_COLORS = {
    0: (255, 165, 0),    # Car GT - Orange
    1: (0, 255, 255),    # Pedestrian GT - Cyan
}

MODEL_TO_OUR_CLASS = {0: 0, 1: 1}


# ============================================================================
# ENHANCED HUNGARIAN MATCHER
# ============================================================================

class EnhancedHungarianMatcher:
    """Enhanced Hungarian matching with IoU + motion + appearance"""
    
    def __init__(self, 
                 iou_weight=0.7,
                 motion_weight=0.2,
                 appearance_weight=0.1,
                 min_iou=0.3,
                 preferred_iou=0.5,
                 img_width=1280,
                 img_height=720):
        
        self.iou_weight = iou_weight
        self.motion_weight = motion_weight
        self.appearance_weight = appearance_weight
        self.min_iou = min_iou
        self.preferred_iou = preferred_iou
        self.img_width = img_width
        self.img_height = img_height
        
        # Track history for motion prediction
        self.track_history = defaultdict(list)
    
    def compute_iou(self, box1: np.ndarray, box2: np.ndarray) -> float:
        """Compute IoU between two boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        inter = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - inter
        
        return inter / union if union > 0 else 0.0
    
    def compute_motion_cost(self, 
                           gt_box: np.ndarray, 
                           pred_id: int,
                           frame_idx: int) -> float:
        """Compute motion cost based on predicted position"""
        
        history = self.track_history.get(pred_id, [])
        
        if len(history) < 2:
            return 0.0
        
        # Get last two positions
        last_box = history[-1]['box']
        prev_box = history[-2]['box']
        
        # Predict current position based on velocity
        velocity = last_box - prev_box
        predicted_box = last_box + velocity
        
        # Compute distance between GT and predicted position
        gt_center = np.array([(gt_box[0] + gt_box[2])/2, (gt_box[1] + gt_box[3])/2])
        pred_center = np.array([(predicted_box[0] + predicted_box[2])/2, 
                               (predicted_box[1] + predicted_box[3])/2])
        
        distance = np.linalg.norm(gt_center - pred_center)
        
        # Normalize by image diagonal
        diagonal = np.sqrt(self.img_width**2 + self.img_height**2)
        normalized_distance = distance / diagonal
        
        return min(1.0, normalized_distance)
    
    def compute_cost_matrix(self, 
                           gt_objs: list, 
                           pred_objs: list,
                           frame_idx: int,
                           use_motion: bool = True,
                           class_id: Optional[int] = None) -> np.ndarray:
        """Compute enhanced cost matrix with IoU + motion"""
        
        n_gt = len(gt_objs)
        n_pred = len(pred_objs)
        
        if n_gt == 0 or n_pred == 0:
            return np.empty((n_gt, n_pred))
        
        # Initialize cost matrix
        cost_matrix = np.ones((n_gt, n_pred)) * 1e6
        
        for i, gt_obj in enumerate(gt_objs):
            gt_box = np.array(gt_obj['box'])
            gt_class = gt_obj.get('class', 0)
            
            for j, pred_obj in enumerate(pred_objs):
                pred_box = np.array(pred_obj['box'])
                pred_class = pred_obj.get('class', 0)
                pred_id = pred_obj.get('id', -1)
                
                # Skip if different classes (when not filtering by class)
                if class_id is None and gt_class != pred_class:
                    cost_matrix[i, j] = 1e6
                    continue
                
                # 1. IoU cost
                iou = self.compute_iou(gt_box, pred_box)
                
                if iou < self.min_iou:
                    cost_matrix[i, j] = 1e6
                    continue
                
                iou_cost = 1.0 - iou
                
                # Add penalty for below preferred threshold (soft threshold)
                if iou < self.preferred_iou:
                    iou_cost += 0.1 * (self.preferred_iou - iou)
                
                # 2. Motion cost (if using track history)
                motion_cost = 0.0
                if use_motion and pred_id >= 0 and pred_id in self.track_history:
                    motion_cost = self.compute_motion_cost(gt_box, pred_id, frame_idx)
                
                # 3. Appearance cost (placeholder - could add ReID features)
                appearance_cost = 0.0
                
                # Combined cost
                total_cost = (self.iou_weight * iou_cost + 
                            self.motion_weight * motion_cost + 
                            self.appearance_weight * appearance_cost)
                
                cost_matrix[i, j] = total_cost
        
        return cost_matrix
    
    def update_track_history(self, pred_objs: list, frame_idx: int):
        """Update track history for motion prediction"""
        for pred_obj in pred_objs:
            pred_id = pred_obj.get('id', -1)
            if pred_id >= 0:
                self.track_history[pred_id].append({
                    'frame': frame_idx,
                    'box': np.array(pred_obj['box'])
                })
                
                # Keep only last 10 frames
                if len(self.track_history[pred_id]) > 10:
                    self.track_history[pred_id].pop(0)
    
    def match(self, gt_objs: list, pred_objs: list, frame_idx: int, class_id: Optional[int] = None):
        """Perform enhanced Hungarian matching"""
        
        # Compute enhanced cost matrix
        cost_matrix = self.compute_cost_matrix(gt_objs, pred_objs, frame_idx, 
                                               use_motion=True, class_id=class_id)
        
        if cost_matrix.size == 0:
            return [], list(range(len(gt_objs))), list(range(len(pred_objs)))
        
        # Hungarian algorithm
        gt_indices, pred_indices = linear_sum_assignment(cost_matrix)
        
        # Filter out invalid matches
        matched = []
        unmatched_gt = set(range(len(gt_objs)))
        unmatched_pred = set(range(len(pred_objs)))
        
        for gt_idx, pred_idx in zip(gt_indices, pred_indices):
            if cost_matrix[gt_idx, pred_idx] < 1e5:  # Valid match
                matched.append((gt_idx, pred_idx))
                unmatched_gt.discard(gt_idx)
                unmatched_pred.discard(pred_idx)
        
        # Update track history
        self.update_track_history(pred_objs, frame_idx)
        
        return matched, list(unmatched_gt), list(unmatched_pred)


# ============================================================================
# DETECTION METRICS COMPUTATION (TP, FP, FN)
# ============================================================================

def compute_detection_metrics(gt_frames: Dict, pred_frames: Dict, 
                              class_id: Optional[int] = None,
                              iou_threshold: float = 0.5,
                              img_width: int = 1280,
                              img_height: int = 720) -> Dict:
    """
    Compute TP, FP, FN for detection task.
    
    TP: Predictions that match GT boxes (IoU >= threshold)
    FP: Predictions with no matching GT box
    FN: GT boxes with no matching prediction
    """
    
    # Filter by class if specified
    if class_id is not None:
        gt_frames_filt = {}
        pred_frames_filt = {}
        
        for frame_idx, objs in gt_frames.items():
            gt_frames_filt[frame_idx] = [obj for obj in objs if obj['class'] == class_id]
        
        for frame_idx, objs in pred_frames.items():
            pred_frames_filt[frame_idx] = [obj for obj in objs if obj['class'] == class_id]
        
        gt_frames = gt_frames_filt
        pred_frames = pred_frames_filt
    
    # Initialize matcher
    matcher = EnhancedHungarianMatcher(
        iou_weight=0.7,
        motion_weight=0.2,
        appearance_weight=0.1,
        min_iou=max(0.3, iou_threshold),
        preferred_iou=0.5,
        img_width=img_width,
        img_height=img_height
    )
    
    total_tp = 0
    total_fp = 0
    total_fn = 0
    
    # Get all frame indices
    all_frame_indices = set(gt_frames.keys()) | set(pred_frames.keys())
    
    for frame_idx in sorted(all_frame_indices):
        gt_objs = gt_frames.get(frame_idx, [])
        pred_objs = pred_frames.get(frame_idx, [])
        
        if not gt_objs and not pred_objs:
            continue
        
        # Use enhanced matching
        matched, unmatched_gt, unmatched_pred = matcher.match(
            gt_objs, pred_objs, frame_idx, class_id=class_id
        )
        
        # TP: matched predictions
        tp = len(matched)
        
        # FP: unmatched predictions (false alarms)
        fp = len(unmatched_pred)
        
        # FN: unmatched GT (missed detections)
        fn = len(unmatched_gt)
        
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    return {
        'TP': total_tp,
        'FP': total_fp,
        'FN': total_fn,
    }


# ============================================================================
# ORIGINAL FUNCTIONS (kept for compatibility)
# ============================================================================

def load_ground_truth_labels(labels_csv: str) -> pd.DataFrame:
    """
    Load ground truth labels from CSV file as a DataFrame.
    Returns:
        pd.DataFrame: The loaded DataFrame with all labels.
    """
    print(f"[INFO] Loading ground truth labels from: {labels_csv}")
    
    # Load the entire CSV as a DataFrame
    df = pd.read_csv(labels_csv, dtype={'videoName': str})
    
    # Add a 'class_id' column required by other functions (assuming you map 'category' 
    # based on your global CLASS_NAMES: ["car", "pedestrian"])
    CATEGORY_TO_ID = {name: i for i, name in enumerate(CLASS_NAMES)}
    df['class_id'] = df['category'].map(CATEGORY_TO_ID)
    
    print(f"[INFO] Loaded labels for {df['videoName'].nunique()} videos")
    print(f"[INFO] Sample video names: {list(df['videoName'].unique())[:5]}")
    
    return df


def find_image_sequence(video_name: str, video_dir: str) -> Optional[List[str]]:
    """Find image sequence for a video."""
    video_folder = os.path.join(video_dir, video_name)
    
    if not os.path.exists(video_folder) or not os.path.isdir(video_folder):
        return None
    
    images = sorted(Path(video_folder).glob("*.jpg"))
    if not images:
        images = sorted(Path(video_folder).glob("*.png"))
    
    return [str(f) for f in images] if images else None


def get_validation_videos(video_dir: str, gt_labels: Dict, seed: int = 42) -> List[Tuple[str, List[str]]]:
    """Get validation videos (same split as training)."""
    
    print(f"\n{'='*80}")
    print(f"SELECTING VALIDATION VIDEOS")
    print(f"{'='*80}\n")
    
    available_videos = []
    for video_name in sorted(gt_labels.keys()):
        images = find_image_sequence(video_name, video_dir)
        if images:
            available_videos.append(video_name)
    
    print(f"[INFO] Found {len(available_videos)} total videos")
    
    import random
    random.seed(seed)
    random.shuffle(available_videos)
    
    val_videos = available_videos[150:200]
    
    print(f"[INFO] Using {len(val_videos)} validation videos")
    
    videos_with_sequences = []
    for video_name in val_videos:
        images = find_image_sequence(video_name, video_dir)
        if images:
            videos_with_sequences.append((video_name, images))
    
    print(f"[INFO] Found {len(videos_with_sequences)} validation videos with images")
    
    return videos_with_sequences


# ============================================================================
# ENHANCED HOTA COMPUTATION
# ============================================================================

def compute_hota_enhanced(gt_frames: Dict, pred_frames: Dict, 
                         class_id: Optional[int] = None,
                         iou_threshold: float = 0.5,
                         img_width: int = 1280,
                         img_height: int = 720) -> Dict:
    """Compute HOTA with enhanced Hungarian matching"""
    
    # Filter by class if specified
    if class_id is not None:
        gt_frames_filt = {}
        pred_frames_filt = {}
        
        for frame_idx, objs in gt_frames.items():
            gt_frames_filt[frame_idx] = [obj for obj in objs if obj['class'] == class_id]
        
        for frame_idx, objs in pred_frames.items():
            pred_frames_filt[frame_idx] = [obj for obj in objs if obj['class'] == class_id]
        
        gt_frames = gt_frames_filt
        pred_frames = pred_frames_filt
    
    if not any(gt_frames.values()) and not any(pred_frames.values()):
        return {'HOTA': 0.0, 'DetA': 0.0, 'AssA': 0.0, 'DetRe': 0.0, 'DetPr': 0.0, 'LocA': 0.0}
    
    # Initialize enhanced matcher
    matcher = EnhancedHungarianMatcher(
        iou_weight=0.7,
        motion_weight=0.2,
        appearance_weight=0.1,
        min_iou=0.3,
        preferred_iou=0.5,
        img_width=img_width,
        img_height=img_height
    )
    
    total_tp = 0
    total_fp = 0
    total_fn = 0
    total_matches = 0
    total_iou_sum = 0.0
    
    gt_track_to_pred = defaultdict(set)
    pred_track_to_gt = defaultdict(set)
    
    for frame_idx in sorted(gt_frames.keys()):
        gt_objs = gt_frames.get(frame_idx, [])
        pred_objs = pred_frames.get(frame_idx, [])
        
        if not gt_objs and not pred_objs:
            continue
        
        # Use enhanced matching
        matched, unmatched_gt, unmatched_pred = matcher.match(
            gt_objs, pred_objs, frame_idx, class_id=class_id
        )
        
        # Process matches
        for gt_idx, pred_idx in matched:
            gt_obj = gt_objs[gt_idx]
            pred_obj = pred_objs[pred_idx]
            
            iou = matcher.compute_iou(
                np.array(gt_obj['box']), 
                np.array(pred_obj['box'])
            )
            
            if iou >= iou_threshold:
                total_iou_sum += iou
                total_matches += 1
                
                gt_id = gt_obj['id']
                pred_id = pred_obj['id']
                gt_track_to_pred[gt_id].add(pred_id)
                pred_track_to_gt[pred_id].add(gt_id)
        
        tp = len(matched)
        fp = len(unmatched_pred)
        fn = len(unmatched_gt)
        
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    # Compute metrics
    det_re = total_tp / max(1, total_tp + total_fn)
    det_pr = total_tp / max(1, total_tp + total_fp)
    det_a = (det_re * det_pr) / max(1e-10, det_re + det_pr - det_re * det_pr)
    
    loc_a = total_iou_sum / max(1, total_matches)
    
    correct_associations = 0
    for gt_id, pred_set in gt_track_to_pred.items():
        if len(pred_set) == 1:
            pred_id = list(pred_set)[0]
            if len(pred_track_to_gt[pred_id]) == 1:
                correct_associations += 1
    
    total_gt_tracks = len(gt_track_to_pred)
    total_pred_tracks = len(pred_track_to_gt)
    
    ass_re = correct_associations / max(1, total_gt_tracks)
    ass_pr = correct_associations / max(1, total_pred_tracks)
    ass_a = (ass_re * ass_pr) / max(1e-10, ass_re + ass_pr - ass_re * ass_pr)
    
    hota = np.sqrt(det_a * ass_a)
    
    return {
        'HOTA': hota * 100,
        'DetA': det_a * 100,
        'AssA': ass_a * 100,
        'DetRe': det_re * 100,
        'DetPr': det_pr * 100,
        'LocA': loc_a * 100,
    }


def compute_iou(box1: np.ndarray, box2: np.ndarray) -> float:
    """Compute IoU between two boxes."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter
    
    return inter / union if union > 0 else 0.0


def compute_mot_metrics_classwise(gt_df: pd.DataFrame, predictions: Dict, class_id: int) -> Dict:
    """Compute MOT metrics for specific class."""
    
    gt_df_cls = gt_df[gt_df['class_id'] == class_id]
    
    gt_frames = defaultdict(list)
    gt_indices = set()
    
    for _, row in gt_df_cls.iterrows():
        frame_idx = int(row['frameIndex'])
        gt_indices.add(frame_idx)
        gt_frames[frame_idx].append({
            'id': int(row['id']),
            'box': [row['box2d.x1'], row['box2d.y1'], row['box2d.x2'], row['box2d.y2']],
            'class': int(row['class_id'])
        })
    
    pred_frames = defaultdict(list)
    for frame_idx in gt_indices:
        if frame_idx in predictions:
            for det in predictions[frame_idx]:
                if det['class_id'] == class_id:
                    pred_frames[frame_idx].append({
                        'id': det['track_id'],
                        'box': [det['x1'], det['y1'], det['x2'], det['y2']],
                        'class': det['class_id']
                    })
    
    acc = mm.MOTAccumulator(auto_id=True)
    
    for frame_idx in sorted(gt_indices):
        gt_objs = gt_frames[frame_idx]
        pred_objs = pred_frames[frame_idx]
        
        gt_ids = [obj['id'] for obj in gt_objs]
        pred_ids = [obj['id'] for obj in pred_objs]
        
        distances = []
        for gt_obj in gt_objs:
            row = []
            for pred_obj in pred_objs:
                iou = compute_iou(np.array(gt_obj['box']), np.array(pred_obj['box']))
                row.append(1.0 - iou)
            distances.append(row)
        
        distances = np.array(distances) if distances else np.empty((0, 0))
        acc.update(gt_ids, pred_ids, distances)
    
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['mota', 'motp', 'num_switches', 
                                       'num_matches', 'num_false_positives', 
                                       'num_misses', 'precision', 'recall'], 
                        name='video')
    idf1 = mh.compute(acc, metrics=['idf1'], name='video')
    
    result = {
        'MOTA': summary['mota'].values[0] * 100 if not summary.empty else 0,
        'MOTP': (1 - summary['motp'].values[0]) * 100 if not summary.empty else 0,
        'IDF1': idf1['idf1'].values[0] * 100 if not idf1.empty else 0,
        'Precision': summary['precision'].values[0] * 100 if not summary.empty else 0,
        'Recall': summary['recall'].values[0] * 100 if not summary.empty else 0,
        'Num_Switches': int(summary['num_switches'].values[0]) if not summary.empty else 0,
        'GT_Objects': len(gt_df_cls),
        'GT_Tracks': gt_df_cls['id'].nunique(),
    }
    
    return result


def compute_mot_metrics(gt_df: pd.DataFrame, predictions: Dict, img_width: int = 1280, img_height: int = 720) -> Dict:
    """Compute MOT metrics with enhanced matching + detection metrics (TP, FP, FN)."""
    
    # Prepare frames
    gt_frames = defaultdict(list)
    gt_indices = set()
    
    for _, row in gt_df.iterrows():
        frame_idx = int(row['frameIndex'])
        gt_indices.add(frame_idx)
        gt_frames[frame_idx].append({
            'id': int(row['id']),
            'box': [row['box2d.x1'], row['box2d.y1'], row['box2d.x2'], row['box2d.y2']],
            'class': int(row['class_id'])
        })
    
    pred_frames = defaultdict(list)
    for frame_idx in gt_indices:
        if frame_idx in predictions:
            for det in predictions[frame_idx]:
                pred_frames[frame_idx].append({
                    'id': det['track_id'],
                    'box': [det['x1'], det['y1'], det['x2'], det['y2']],
                    'class': det['class_id']
                })
    
    # Overall MOT metrics (still use motmetrics for compatibility)
    acc = mm.MOTAccumulator(auto_id=True)
    
    for frame_idx in sorted(gt_indices):
        gt_objs = gt_frames[frame_idx]
        pred_objs = pred_frames[frame_idx]
        
        gt_ids = [obj['id'] for obj in gt_objs]
        pred_ids = [obj['id'] for obj in pred_objs]
        
        distances = []
        for gt_obj in gt_objs:
            row = []
            for pred_obj in pred_objs:
                if gt_obj['class'] != pred_obj['class']:
                    row.append(np.nan)
                else:
                    iou = compute_iou(np.array(gt_obj['box']), np.array(pred_obj['box']))
                    row.append(1.0 - iou)
            distances.append(row)
        
        distances = np.array(distances) if distances else np.empty((0, 0))
        acc.update(gt_ids, pred_ids, distances)
    
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['mota', 'motp', 'num_switches', 
                                       'num_matches', 'num_false_positives', 
                                       'num_misses', 'precision', 'recall'], 
                        name='video')
    idf1 = mh.compute(acc, metrics=['idf1'], name='video')
    
    # Overall HOTA with enhanced matching
    hota_metrics = compute_hota_enhanced(gt_frames, pred_frames, class_id=None, 
                                        iou_threshold=0.5, img_width=img_width, img_height=img_height)
    
    # Overall detection metrics (TP, FP, FN)
    detection_metrics = compute_detection_metrics(gt_frames, pred_frames, class_id=None,
                                                  iou_threshold=0.5, img_width=img_width, img_height=img_height)
    
    result = {
        'MOTA': summary['mota'].values[0] * 100 if not summary.empty else 0,
        'MOTP': (1 - summary['motp'].values[0]) * 100 if not summary.empty else 0,
        'IDF1': idf1['idf1'].values[0] * 100 if not idf1.empty else 0,
        'Precision': summary['precision'].values[0] * 100 if not summary.empty else 0,
        'Recall': summary['recall'].values[0] * 100 if not summary.empty else 0,
        'Num_Switches': int(summary['num_switches'].values[0]) if not summary.empty else 0,
        # Add detection metrics
        'TP': detection_metrics['TP'],
        'FP': detection_metrics['FP'],
        'FN': detection_metrics['FN'],
    }
    
    result.update(hota_metrics)
    
    # Class-wise metrics with enhanced matching + detection metrics
    for class_id, class_name in enumerate(CLASS_NAMES):
        # HOTA class-wise with enhanced matching
        hota_cls = compute_hota_enhanced(gt_frames, pred_frames, class_id, 
                                        iou_threshold=0.5, img_width=img_width, img_height=img_height)
        for metric_name, metric_val in hota_cls.items():
            result[f'{class_name}_{metric_name}'] = metric_val
        
        # MOT class-wise
        mot_cls = compute_mot_metrics_classwise(gt_df, predictions, class_id)
        for metric_name, metric_val in mot_cls.items():
            result[f'{class_name}_{metric_name}'] = metric_val
        
        # Detection metrics class-wise (TP, FP, FN)
        det_cls = compute_detection_metrics(gt_frames, pred_frames, class_id,
                                           iou_threshold=0.5, img_width=img_width, img_height=img_height)
        for metric_name, metric_val in det_cls.items():
            result[f'{class_name}_{metric_name}'] = metric_val
    
    return result


def draw_box_with_label(frame: np.ndarray, x1: int, y1: int, x2: int, y2: int, 
                        label: str, color: Tuple[int, int, int], thickness: int = 2):
    """Draw a box with label on frame."""
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
    
    label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    label_y = max(y1 - 10, label_size[1] + 10)
    
    cv2.rectangle(frame, 
                 (x1, label_y - label_size[1] - 5), 
                 (x1 + label_size[0] + 5, label_y + 5), 
                 color, -1)
    
    cv2.putText(frame, label, (x1 + 2, label_y), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)


def process_video(video_name: str, image_paths: List[str], model: YOLO, 
                 gt_df: pd.DataFrame, tracker: str, conf: float, iou: float,
                 save_video: bool = False, output_dir: str = None) -> Dict:
    """Process video with tracking and visualization."""
    
    print(f"\n{'='*80}")
    print(f"VIDEO: {video_name}")
    print(f"{'='*80}")
    
    first_img = cv2.imread(image_paths[0])
    if first_img is None:
        print(f"[ERROR] Cannot read {image_paths[0]}")
        return {}
    
    height, width = first_img.shape[:2]
    
    gt_by_frame = defaultdict(list)
    for _, row in gt_df.iterrows():
        frame_idx = int(row['frameIndex'])
        gt_by_frame[frame_idx].append({
            'id': int(row['id']),
            'class_id': int(row['class_id']),
            'x1': row['box2d.x1'],
            'y1': row['box2d.y1'],
            'x2': row['box2d.x2'],
            'y2': row['box2d.y2']
        })
    
    gt_indices = set(gt_by_frame.keys())
    
    class_counts = gt_df['class_id'].value_counts().to_dict()
    car_count = class_counts.get(0, 0)
    ped_count = class_counts.get(1, 0)
    
    print(f"  Dimensions: {width}x{height}")
    print(f"  Total frames: {len(image_paths)}")
    print(f"  GT frames: {len(gt_indices)}")
    print(f"  GT Cars: {car_count} objects, {gt_df[gt_df['class_id']==0]['id'].nunique()} tracks")
    print(f"  GT Pedestrians: {ped_count} objects, {gt_df[gt_df['class_id']==1]['id'].nunique()} tracks")
    
    video_writer = None
    if save_video and output_dir:
        vis_dir = os.path.join(output_dir, "tracking_videos")
        os.makedirs(vis_dir, exist_ok=True)
        video_out_path = os.path.join(vis_dir, f"{video_name}_tracking.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(video_out_path, fourcc, 30.0, (width, height))
    
    predictions = {}
    unique_tracks = set()
    unique_tracks_by_class = {0: set(), 1: set()}
    
    for frame_idx, img_path in enumerate(tqdm(image_paths, desc=f"  Tracking", ncols=100, leave=False)):
        frame = cv2.imread(img_path)
        if frame is None:
            continue
        
        try:
            results = model.track(frame, persist=True, tracker=tracker, 
                                conf=conf, iou=iou, verbose=False)
        except:
            continue
        
        frame_dets = []
        if results and results[0].boxes is not None and results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            confidences = results[0].boxes.conf.cpu().numpy()
            class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
            
            for box, tid, conf_val, cls in zip(boxes, track_ids, confidences, class_ids):
                our_cls = MODEL_TO_OUR_CLASS.get(int(cls))
                if our_cls is None:
                    continue
                
                x1, y1, x2, y2 = box
                frame_dets.append({
                    'class_id': our_cls,
                    'track_id': int(tid),
                    'confidence': float(conf_val),
                    'x1': float(x1), 'y1': float(y1), 'x2': float(x2), 'y2': float(y2)
                })
                unique_tracks.add(int(tid))
                unique_tracks_by_class[our_cls].add(int(tid))
        
        if frame_idx in gt_indices:
            predictions[frame_idx] = frame_dets
        
        if video_writer is not None:
            vis_frame = frame.copy()
            
            if frame_idx in gt_by_frame:
                for gt_obj in gt_by_frame[frame_idx]:
                    color = GT_COLORS[gt_obj['class_id']]
                    label = f"GT-{CLASS_NAMES[gt_obj['class_id']]} ID:{gt_obj['id']}"
                    draw_box_with_label(vis_frame, 
                                       int(gt_obj['x1']), int(gt_obj['y1']),
                                       int(gt_obj['x2']), int(gt_obj['y2']),
                                       label, color, thickness=2)
            
            for det in frame_dets:
                color = PRED_COLORS[det['class_id']]
                label = f"PRED-{CLASS_NAMES[det['class_id']]} ID:{det['track_id']}"
                draw_box_with_label(vis_frame,
                                   int(det['x1']), int(det['y1']),
                                   int(det['x2']), int(det['y2']),
                                   label, color, thickness=3)
            
            legend_y = 30
            cv2.putText(vis_frame, f"Frame: {frame_idx}", (10, legend_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            legend_y += 30
            
            if frame_idx in gt_indices:
                cv2.putText(vis_frame, "GT FRAME", (10, legend_y),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            
            legend_x = width - 250
            legend_y = 30
            
            for i, (class_id, class_name) in enumerate([(0, "Car"), (1, "Pedestrian")]):
                cv2.rectangle(vis_frame, (legend_x, legend_y), (legend_x + 20, legend_y + 15), GT_COLORS[class_id], -1)
                cv2.putText(vis_frame, f"GT {class_name}", (legend_x + 25, legend_y + 12),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                legend_y += 20
                
                cv2.rectangle(vis_frame, (legend_x, legend_y), (legend_x + 20, legend_y + 15), PRED_COLORS[class_id], -1)
                cv2.putText(vis_frame, f"Pred {class_name}", (legend_x + 25, legend_y + 12),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                legend_y += 25
            
            video_writer.write(vis_frame)
    
    if video_writer is not None:
        video_writer.release()
        print(f"  ✅ Video saved")
    
    print(f"\n  Computing metrics with enhanced matching...")
    metrics = compute_mot_metrics(gt_df, predictions, img_width=width, img_height=height)
    metrics['video_name'] = video_name
    metrics['unique_tracks'] = len(unique_tracks)
    metrics['car_tracks'] = len(unique_tracks_by_class[0])
    metrics['pedestrian_tracks'] = len(unique_tracks_by_class[1])
    
    # Display results
    print(f"\n  {'─'*76}")
    print(f"  RESULTS FOR: {video_name}")
    print(f"  {'─'*76}\n")
    
    print(f"  OVERALL METRICS (Enhanced Matching):")
    print(f"    HOTA:      {metrics.get('HOTA', 0):6.2f}%  |  DetA: {metrics.get('DetA', 0):6.2f}%  |  AssA: {metrics.get('AssA', 0):6.2f}%")
    print(f"    MOTA:      {metrics.get('MOTA', 0):6.2f}%  |  MOTP: {metrics.get('MOTP', 0):6.2f}%  |  IDF1: {metrics.get('IDF1', 0):6.2f}%")
    print(f"    Precision: {metrics.get('Precision', 0):6.2f}%  |  Recall: {metrics.get('Recall', 0):6.2f}%")
    print(f"    ID Switches: {metrics.get('Num_Switches', 0)}")
    print(f"    TP: {metrics.get('TP', 0):5d}  |  FP: {metrics.get('FP', 0):5d}  |  FN: {metrics.get('FN', 0):5d}")
    
    print(f"\n  CAR METRICS (Enhanced Matching):")
    print(f"    HOTA:      {metrics.get('car_HOTA', 0):6.2f}%  |  DetA: {metrics.get('car_DetA', 0):6.2f}%  |  AssA: {metrics.get('car_AssA', 0):6.2f}%")
    print(f"    MOTA:      {metrics.get('car_MOTA', 0):6.2f}%  |  MOTP: {metrics.get('car_MOTP', 0):6.2f}%  |  IDF1: {metrics.get('car_IDF1', 0):6.2f}%")
    print(f"    Precision: {metrics.get('car_Precision', 0):6.2f}%  |  Recall: {metrics.get('car_Recall', 0):6.2f}%")
    print(f"    GT Objects: {metrics.get('car_GT_Objects', 0):4d}  |  GT Tracks: {metrics.get('car_GT_Tracks', 0):3d}  |  Pred Tracks: {metrics.get('car_tracks', 0):3d}")
    print(f"    ID Switches: {metrics.get('car_Num_Switches', 0)}")
    print(f"    TP: {metrics.get('car_TP', 0):5d}  |  FP: {metrics.get('car_FP', 0):5d}  |  FN: {metrics.get('car_FN', 0):5d}")
    
    print(f"\n  PEDESTRIAN METRICS (Enhanced Matching):")
    print(f"    HOTA:      {metrics.get('pedestrian_HOTA', 0):6.2f}%  |  DetA: {metrics.get('pedestrian_DetA', 0):6.2f}%  |  AssA: {metrics.get('pedestrian_AssA', 0):6.2f}%")
    print(f"    MOTA:      {metrics.get('pedestrian_MOTA', 0):6.2f}%  |  MOTP: {metrics.get('pedestrian_MOTP', 0):6.2f}%  |  IDF1: {metrics.get('pedestrian_IDF1', 0):6.2f}%")
    print(f"    Precision: {metrics.get('pedestrian_Precision', 0):6.2f}%  |  Recall: {metrics.get('pedestrian_Recall', 0):6.2f}%")
    print(f"    GT Objects: {metrics.get('pedestrian_GT_Objects', 0):4d}  |  GT Tracks: {metrics.get('pedestrian_GT_Tracks', 0):3d}  |  Pred Tracks: {metrics.get('pedestrian_tracks', 0):3d}")
    print(f"    ID Switches: {metrics.get('pedestrian_Num_Switches', 0)}")
    print(f"    TP: {metrics.get('pedestrian_TP', 0):5d}  |  FP: {metrics.get('pedestrian_FP', 0):5d}  |  FN: {metrics.get('pedestrian_FN', 0):5d}")
    
    print(f"\n  {'─'*76}")
    print(f"  ✅ Video processing complete")
    print(f"  {'='*80}\n")
    
    return metrics


def main():
    """Main evaluation workflow."""
    
    print(f"\n{'='*80}")
    print(f"BDD100K MOT Evaluation with Enhanced Hungarian Matching")
    print(f"Date: 2025-11-04 13:14:36 UTC")
    print(f"User: manishkolachalamgmail")
    print(f"{'='*80}\n")
    
    print("📊 ENHANCEMENTS:")
    print("  ✅ IoU + Motion + Appearance matching")
    print("  ✅ Soft thresholds (min: 0.3, preferred: 0.5)")
    print("  ✅ Temporal smoothing (10-frame history)")
    print("  ✅ Class-wise metrics")
    print("  ✅ Detection metrics (TP, FP, FN) per video and per class")
    print()
    
    base_dir = r"C:\Users\Manish\Documents\SpikeYOLOALL"
    video_dir = r'C:\Users\Manish\Documents\SpikeYOLOALL\KITTITRACK\data_tracking_image_2\training\image_02'
    labels_csv = r"C:\Users\Manish\Documents\SpikeYOLOALL\KITTITRACK\tracking_bdd_format.csv"
    model_path = 'BDD1280_ep6.pt'
    output_dir = os.path.join(base_dir, "KITTI_validation_tracking")
    
    if not os.path.exists(labels_csv):
        print(f"[ERROR] CSV not found: {labels_csv}")
        return
    
    if not os.path.exists(model_path):
        print(f"[ERROR] Model not found: {model_path}")
        return
    
    print(f"[INFO] Loading fine-tuned model: {model_path}")
    model = YOLO(model_path)
    
    # *** FIX 1: Load the full GT DataFrame, not a nested dict ***
    full_gt_df = load_ground_truth_labels(labels_csv)
    
    # Get all unique video names from the DataFrame
    all_video_names = full_gt_df['videoName'].unique()
    
    # *** FIX 2: Correctly generate the list of (video_name, image_paths) tuples ***
    # This replaces the need for get_validation_videos() if we process all available videos
    
    videos_to_process = []
    print("\n[INFO] Finding image sequences for videos...")
    for video_name in all_video_names:
        image_paths = find_image_sequence(video_name, video_dir)
        if image_paths:
            videos_to_process.append((video_name, image_paths))
        else:
            print(f"  [WARN] Image sequence not found for: {video_name}")

    if not videos_to_process:
        print("[ERROR] No validation videos with image sequences found")
        return
    
    tracker = "botsort.yaml"
    conf_threshold = 0.01
    iou_threshold = 0.5
    
    print(f"\n[INFO] Configuration:")
    print(f"  Tracker: {tracker}")
    print(f"  Confidence threshold: {conf_threshold}")
    print(f"  IoU threshold: {iou_threshold}")
    print(f"  Enhanced matching: ENABLED")
    
    save_vids = input("\nSave visualization videos? (y/n) [default: y]: ").strip().lower()
    save_vids = save_vids != 'n'
    
    os.makedirs(output_dir, exist_ok=True)
    all_stats = []
    
    # Use the correctly constructed list of tuples
    print(f"\n{'='*80}")
    print(f"PROCESSING {len(videos_to_process)} VALIDATION VIDEOS")
    print(f"{'='*80}\n")
    
    for video_idx, (video_name, image_paths) in enumerate(videos_to_process, 1):
        print(f"\n[{video_idx}/{len(videos_to_process)}]")
        
        # *** FIX 3: Filter the full DataFrame to get the video's GT DataFrame ***
        gt_df = full_gt_df[full_gt_df['videoName'] == video_name].copy()
        
        stats = process_video(
            video_name, image_paths, model, gt_df,
            tracker, conf_threshold, iou_threshold,
            save_video=save_vids, output_dir=output_dir
        )
        
        if stats:
            all_stats.append(stats)
    
    if all_stats:
        df = pd.DataFrame(all_stats)
        csv_path = os.path.join(output_dir, "validation_tracking_results_enhanced.csv")
        df.to_csv(csv_path, index=False)
        
        print(f"\n{'='*80}")
        print(f"AGGREGATE RESULTS ({len(all_stats)} videos)")
        print(f"{'='*80}\n")
        
        print("OVERALL METRICS (All Classes - Enhanced Matching):")
        for metric in ['HOTA', 'MOTA', 'MOTP', 'IDF1', 'DetA', 'AssA', 'Precision', 'Recall']:
            if metric in df.columns:
                mean_val = df[metric].mean()
                std_val = df[metric].std()
                min_val = df[metric].min()
                max_val = df[metric].max()
                print(f"  {metric:12s}: {mean_val:6.2f}% (±{std_val:5.2f}%) | Range: [{min_val:6.2f}%, {max_val:6.2f}%]")
        
        print(f"\nOVERALL DETECTION METRICS:")
        for metric in ['TP', 'FP', 'FN']:
            if metric in df.columns:
                total_val = df[metric].sum()
                mean_val = df[metric].mean()
                std_val = df[metric].std()
                print(f"  {metric:12s}: {total_val:7d} total | {mean_val:6.2f} (±{std_val:5.2f}) avg/video")
        
        print(f"\nCAR METRICS (Enhanced Matching):")
        for metric in ['HOTA', 'MOTA', 'MOTP', 'IDF1', 'DetA', 'AssA', 'Precision', 'Recall']:
            col = f'car_{metric}'
            if col in df.columns:
                mean_val = df[col].mean()
                std_val = df[col].std()
                print(f"  {metric:12s}: {mean_val:6.2f}% (±{std_val:5.2f}%)")
        
        if 'car_Num_Switches' in df.columns:
            print(f"  ID Switches:  {df['car_Num_Switches'].sum():4d} total, {df['car_Num_Switches'].mean():5.2f} avg/video")
        
        print(f"\nCAR DETECTION METRICS:")
        for metric in ['TP', 'FP', 'FN']:
            col = f'car_{metric}'
            if col in df.columns:
                total_val = df[col].sum()
                mean_val = df[col].mean()
                print(f"  {metric:12s}: {total_val:7d} total | {mean_val:6.2f} avg/video")
        
        print(f"\nPEDESTRIAN METRICS (Enhanced Matching):")
        for metric in ['HOTA', 'MOTA', 'MOTP', 'IDF1', 'DetA', 'AssA', 'Precision', 'Recall']:
            col = f'pedestrian_{metric}'
            if col in df.columns:
                mean_val = df[col].mean()
                std_val = df[col].std()
                print(f"  {metric:12s}: {mean_val:6.2f}% (±{std_val:5.2f}%)")
        
        if 'pedestrian_Num_Switches' in df.columns:
            print(f"  ID Switches:  {df['pedestrian_Num_Switches'].sum():4d} total, {df['pedestrian_Num_Switches'].mean():5.2f} avg/video")
        
        print(f"\nPEDESTRIAN DETECTION METRICS:")
        for metric in ['TP', 'FP', 'FN']:
            col = f'pedestrian_{metric}'
            if col in df.columns:
                total_val = df[col].sum()
                mean_val = df[col].mean()
                print(f"  {metric:12s}: {total_val:7d} total | {mean_val:6.2f} avg/video")
        
        print(f"\n{'='*80}")
        print(f"✅ Results saved to: {csv_path}")
        
        if save_vids:
            print(f"📹 Videos saved to: {os.path.join(output_dir, 'tracking_videos')}")
        
        print(f"\n💡 ENHANCEMENT SUMMARY:")
        print(f"  - IoU weight: 70% (primary factor)")
        print(f"  - Motion weight: 20% (temporal consistency)")
        print(f"  - Soft threshold: min 0.3, preferred 0.5")
        print(f"  - Track history: 10 frames")
        print(f"  - Detection metrics: TP, FP, FN computed per video and per class")
        print(f"{'='*80}\n")


if __name__ == "__main__":
    main()


BDD100K MOT Evaluation with Enhanced Hungarian Matching
Date: 2025-11-04 13:14:36 UTC
User: manishkolachalamgmail

📊 ENHANCEMENTS:
  ✅ IoU + Motion + Appearance matching
  ✅ Soft thresholds (min: 0.3, preferred: 0.5)
  ✅ Temporal smoothing (10-frame history)
  ✅ Class-wise metrics
  ✅ Detection metrics (TP, FP, FN) per video and per class

[INFO] Loading fine-tuned model: BDD1280_ep6.pt
[INFO] Loading ground truth labels from: C:\Users\Manish\Documents\SpikeYOLOALL\KITTITRACK\tracking_bdd_format.csv
[INFO] Loaded labels for 21 videos
[INFO] Sample video names: ['0000', '0001', '0002', '0003', '0004']

[INFO] Finding image sequences for videos...

[INFO] Configuration:
  Tracker: botsort.yaml
  Confidence threshold: 0.01
  IoU threshold: 0.5
  Enhanced matching: ENABLED



Save visualization videos? (y/n) [default: y]:  y



PROCESSING 21 VALIDATION VIDEOS


[1/21]

VIDEO: 0000
  Dimensions: 1242x375
  Total frames: 154
  GT frames: 154
  GT Cars: 535 objects, 12 tracks
  GT Pedestrians: 22 objects, 2 tracks


  ✅ Video saved

  Computing metrics with enhanced matching...

  ────────────────────────────────────────────────────────────────────────────
  RESULTS FOR: 0000
  ────────────────────────────────────────────────────────────────────────────

  OVERALL METRICS (Enhanced Matching):
    HOTA:       61.09%  |  DetA:  49.76%  |  AssA:  75.00%
    MOTA:       12.93%  |  MOTP:  69.28%  |  IDF1:  69.10%
    Precision:  53.84%  |  Recall:  99.46%
    ID Switches: 7
    TP:   493  |  FP:   536  |  FN:    64

  CAR METRICS (Enhanced Matching):
    HOTA:       65.21%  |  DetA:  59.54%  |  AssA:  71.43%
    MOTA:       40.00%  |  MOTP:  70.66%  |  IDF1:  76.23%
    Precision:  62.96%  |  Recall:  99.44%
    GT Objects:  535  |  GT Tracks:  12  |  Pred Tracks:  30
    ID Switches: 5
    TP:   487  |  FP:   358  |  FN:    48

  PEDESTRIAN METRICS (Enhanced Matching):
    HOTA:       24.87%  |  DetA:   6.19%  |  AssA: 100.00%
    MOTA:        9.09%  |  MOTP:  35.84%  |  IDF1:  70.97%
    Precision:  

  ✅ Video saved

  Computing metrics with enhanced matching...

  ────────────────────────────────────────────────────────────────────────────
  RESULTS FOR: 0001
  ────────────────────────────────────────────────────────────────────────────

  OVERALL METRICS (Enhanced Matching):
    HOTA:       67.95%  |  DetA:  61.38%  |  AssA:  75.23%
    MOTA:       56.81%  |  MOTP:  53.99%  |  IDF1:  77.76%
    Precision:  71.68%  |  Recall:  97.81%
    ID Switches: 71
    TP:  2585  |  FP:  1522  |  FN:   425

  CAR METRICS (Enhanced Matching):
    HOTA:       67.84%  |  DetA:  61.75%  |  AssA:  74.53%
    MOTA:       57.45%  |  MOTP:  53.16%  |  IDF1:  78.24%
    Precision:  71.73%  |  Recall:  98.86%
    GT Objects: 2898  |  GT Tracks:  94  |  Pred Tracks: 138
    ID Switches: 71
    TP:  2530  |  FP:  1464  |  FN:   368

  PEDESTRIAN METRICS (Enhanced Matching):
    HOTA:       71.42%  |  DetA:  51.01%  |  AssA: 100.00%
    MOTA:       61.61%  |  MOTP:  61.79%  |  IDF1:  71.00%
    Precision:

...

  ✅ Video saved

  Computing metrics with enhanced matching...

  ────────────────────────────────────────────────────────────────────────────
  RESULTS FOR: 0016
  ────────────────────────────────────────────────────────────────────────────

  OVERALL METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:  2863

  CAR METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    GT Objects:  836  |  GT Tracks:   4  |  Pred Tracks:   0
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:   836

  PEDESTRIAN METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:  

  ✅ Video saved

  Computing metrics with enhanced matching...

  ────────────────────────────────────────────────────────────────────────────
  RESULTS FOR: 0017
  ────────────────────────────────────────────────────────────────────────────

  OVERALL METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:   782

  CAR METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:         nan%  |  MOTP:    nan%  |  IDF1:    nan%
    Precision:    nan%  |  Recall:    nan%
    GT Objects:    0  |  GT Tracks:   0  |  Pred Tracks:   0
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:     0

  PEDESTRIAN METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:  

  ✅ Video saved

  Computing metrics with enhanced matching...

  ────────────────────────────────────────────────────────────────────────────
  RESULTS FOR: 0018
  ────────────────────────────────────────────────────────────────────────────

  OVERALL METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:  1413

  CAR METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    GT Objects: 1413  |  GT Tracks:  21  |  Pred Tracks:   0
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:  1413

  PEDESTRIAN METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:         nan%  |  MOTP:    nan%  |  IDF1:    nan%
    Precision:  

  ✅ Video saved

  Computing metrics with enhanced matching...

  ────────────────────────────────────────────────────────────────────────────
  RESULTS FOR: 0019
  ────────────────────────────────────────────────────────────────────────────

  OVERALL METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:  8427

  CAR METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    GT Objects: 1830  |  GT Tracks:  15  |  Pred Tracks:   0
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:  1830

  PEDESTRIAN METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:  

  ✅ Video saved

  Computing metrics with enhanced matching...

  ────────────────────────────────────────────────────────────────────────────
  RESULTS FOR: 0020
  ────────────────────────────────────────────────────────────────────────────

  OVERALL METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:  6404

  CAR METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:        0.00%  |  MOTP:    nan%  |  IDF1:   0.00%
    Precision:    nan%  |  Recall:   0.00%
    GT Objects: 6404  |  GT Tracks: 127  |  Pred Tracks:   0
    ID Switches: 0
    TP:     0  |  FP:     0  |  FN:  6404

  PEDESTRIAN METRICS (Enhanced Matching):
    HOTA:        0.00%  |  DetA:   0.00%  |  AssA:   0.00%
    MOTA:         nan%  |  MOTP:    nan%  |  IDF1:    nan%
    Precision:  

In [4]:
import os
import pandas as pd
from pathlib import Path

# UPDATE THESE PATHS
VIDEO_DIR = r"C:\Users\Manish\Documents\SpikeYOLOALL\KITTITRACK\images"  # UPDATE THIS
LABELS_CSV = r"C:\Users\Manish\Documents\SpikeYOLOALL\KITTITRACK\tracking_bdd_format.csv"

print("="*80)
print("DEBUGGING VIDEO DIRECTORY STRUCTURE")
print("="*80)

# Check CSV
print("\n1. CHECKING CSV:")
df = pd.read_csv(LABELS_CSV, dtype={'videoName': str})
video_names_in_csv = sorted(df['videoName'].unique())
print(f"   Video names in CSV: {video_names_in_csv[:10]}")
print(f"   Total unique videos in CSV: {len(video_names_in_csv)}")

# Check video directory
print(f"\n2. CHECKING VIDEO DIRECTORY: {VIDEO_DIR}")
if not os.path.exists(VIDEO_DIR):
    print(f"   [ERROR] Directory does not exist!")
else:
    print(f"   [OK] Directory exists")
    
    # List what's in the directory
    items = os.listdir(VIDEO_DIR)
    print(f"   Total items in directory: {len(items)}")
    
    # Show first 20 items
    print(f"\n   First 20 items:")
    for i, item in enumerate(sorted(items)[:20]):
        item_path = os.path.join(VIDEO_DIR, item)
        if os.path.isdir(item_path):
            # Count files in subdirectory
            try:
                files_in_subdir = os.listdir(item_path)
                print(f"   [{i:2d}] 📁 {item}/ ({len(files_in_subdir)} files)")
            except:
                print(f"   [{i:2d}] 📁 {item}/ (cannot access)")
        else:
            print(f"   [{i:2d}] 📄 {item}")

# Check for matching patterns
print("\n3. CHECKING FOR MATCHES:")
print("\n   Testing different naming patterns:")

patterns_to_test = [
    lambda x: x,                    # Direct match: "0000"
    lambda x: f"{x}.txt",          # With .txt: "0000.txt"
    lambda x: f"video_{x}",        # With prefix: "video_0000"
    lambda x: x.zfill(4),          # Zero-padded: "0000"
    lambda x: f"{int(x):04d}",     # Integer padded: "0000"
]

pattern_names = [
    "Direct (e.g., '0000')",
    "With .txt (e.g., '0000.txt')",
    "With prefix (e.g., 'video_0000')",
    "Zero-padded (e.g., '0000')",
    "Int-padded (e.g., '0000')"
]

for pattern_name, pattern_func in zip(pattern_names, patterns_to_test):
    matches = 0
    for video_name in video_names_in_csv[:5]:  # Test first 5
        try:
            test_name = pattern_func(video_name)
            test_path = os.path.join(VIDEO_DIR, test_name)
            if os.path.exists(test_path):
                matches += 1
        except:
            pass
    print(f"   {pattern_name}: {matches}/5 matches")

# Check if images are directly in VIDEO_DIR (not in subfolders)
print("\n4. CHECKING FOR DIRECT IMAGE FILES:")
if os.path.exists(VIDEO_DIR):
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    direct_images = [f for f in os.listdir(VIDEO_DIR) 
                     if any(f.lower().endswith(ext) for ext in image_extensions)]
    print(f"   Found {len(direct_images)} image files directly in VIDEO_DIR")
    if direct_images:
        print(f"   Sample image names: {direct_images[:10]}")

# Suggest fixes
print("\n" + "="*80)
print("SUGGESTIONS:")
print("="*80)

if os.path.exists(VIDEO_DIR):
    items = os.listdir(VIDEO_DIR)
    dirs = [d for d in items if os.path.isdir(os.path.join(VIDEO_DIR, d))]
    files = [f for f in items if os.path.isfile(os.path.join(VIDEO_DIR, f))]
    
    if dirs:
        print(f"\n✓ Found {len(dirs)} subdirectories")
        print("  Your structure looks like: VIDEO_DIR/video_name/images")
        print("\n  SOLUTION 1: Update CSV videoName to match folder names")
        print(f"  Example: '{dirs[0]}' (actual folder) vs '{video_names_in_csv[0]}' (CSV)")
        
        if dirs[0].endswith('.txt'):
            print("\n  Quick fix in code:")
            print("  df['videoName'] = df['videoName'] + '.txt'")
        
    elif files:
        print(f"\n✓ Found {len(files)} files directly in VIDEO_DIR")
        print("  Your structure looks like: VIDEO_DIR/image_files")
        print("\n  SOLUTION 2: Your images are NOT in subfolders")
        print("  You need to reorganize: VIDEO_DIR/video_name/frame_*.jpg")

print("\n" + "="*80)

DEBUGGING VIDEO DIRECTORY STRUCTURE

1. CHECKING CSV:
   Video names in CSV: ['0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009']
   Total unique videos in CSV: 21

2. CHECKING VIDEO DIRECTORY: C:\Users\Manish\Documents\SpikeYOLOALL\KITTITRACK\images
   [ERROR] Directory does not exist!

3. CHECKING FOR MATCHES:

   Testing different naming patterns:
   Direct (e.g., '0000'): 0/5 matches
   With .txt (e.g., '0000.txt'): 0/5 matches
   With prefix (e.g., 'video_0000'): 0/5 matches
   Zero-padded (e.g., '0000'): 0/5 matches
   Int-padded (e.g., '0000'): 0/5 matches

4. CHECKING FOR DIRECT IMAGE FILES:

SUGGESTIONS:

